In [1]:
import sys
sys.path.append('/Users/davison/Software/almeida-semantica')
from src.ConectorBD import *
import pandas as pd
from pandasql import sqldf
import xml.etree.ElementTree as ET

In [8]:
sblgnt_db = '/Users/davison/Software/almeida-semantica/db/sbl.db'

In [3]:
sql = "SELECT * FROM bible"

In [ ]:
sblgnt_bible_df = pd.read_sql_query(sql, ConectorBD(sblgnt_db).con)

In [ ]:
sblgnt_bible_df.head()

,book,chapter,verse,content,nr_sq_livro
0,Matt,1,1,"<verse num=""1"">\n<w pos=""N-"" morph=""----NSF-"" ...",40
1,Matt,1,2,"<verse num=""2"">\n<w pos=""N-"" morph=""----NSM-"" ...",40
2,Matt,1,3,"<verse num=""3"">\n<w pos=""N-"" morph=""----NSM-"" ...",40
3,Matt,1,4,"<verse num=""4"">\n<w pos=""N-"" morph=""----NSM-"" ...",40
4,Matt,1,5,"<verse num=""5"">\n<w pos=""N-"" morph=""----NSM-"" ...",40


In [ ]:
len(sblgnt_bible_df)

7927

In [ ]:
xml_vs_content = sblgnt_bible_df.loc[0, 'content']

In [31]:
for i, value in enumerate(ET.fromstring(xml_vs_content)):
    print(value.attrib.get('pos'), value.attrib.get('morph'), value.attrib.get('lemma'), value.attrib.get('strongs'), value.text, value.tail, i+1)

N- ----NSF- βίβλος 00976 Βίβλος   1
N- ----GSF- γένεσις 01078 γενέσεως   2
N- ----GSM- Ἰησοῦς 02424 Ἰησοῦ   3
N- ----GSM- Χριστός 05547 χριστοῦ   4
N- ----GSM- υἱός 05207 υἱοῦ   5
N- ----GSM- Δαυίδ None Δαυὶδ   6
N- ----GSM- υἱός 05207 υἱοῦ   7
N- ----GSM- Ἀβραάμ 00011 Ἀβραάμ .
 8


In [ ]:
import sqlite3

In [ ]:
def migra_sblgnt_bible_to_words(df):
    for i in range(len(df)):
        for j, value in enumerate(ET.fromstring(df.loc[i, 'content'])):
            book = df.loc[i, 'book']
            chapter = df.loc[i, 'chapter']
            verse = df.loc[i, 'verse']
            pos = value.attrib.get('pos')
            morph = value.attrib.get('morph')
            lemma = value.attrib.get('lemma')
            strongs = value.attrib.get('strongs')
            word = value.text
            punct = value.tail
            word_nr = j+1
            nr_sq_livro = df.loc[i, 'nr_sq_livro']

            insert_into = """
                            INSERT INTO words (book, chapter, verse, pos, morph, lemma, strongs, word, punct, word_nr, nr_sq_livro) 
                            VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')
                        """.format(book, chapter, verse, pos, morph, lemma, strongs, word, punct, word_nr, nr_sq_livro)

            try:
                con = sqlite3.connect(sblgnt_db)
                cursor = con.cursor()
                cursor.execute(insert_into)
                con.commit()
                con.close()
            except Exception as e:
                print(e)
            
            # ConectorBD(sblgnt_db).con.execute(insert_into)

In [ ]:
# migra tabela bible para words (com os campos de content [pos, morph, lemma, strongs, word, punct, word_nr] separados)
# migra_sblgnt_bible_to_words(sblgnt_bible_df)